In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
import os
import numpy as np
import tensorflow as tf

# seed 고정
seed = 15
np.random.seed(seed)
tf.random.set_seed(seed)

# 설정값
BATCH_SIZE = 32
EPOCHS = 20
IMG_HEIGHT = 150
IMG_WIDTH = 150

# 데이터 폴더 경로 (너가 준비한 폴더 구조에 맞춰서)
base_dir = 'dataset'  # 직접 준비한 폴더명
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 데이터 제너레이터
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 흐름 만들기 (flow_from_directory 사용)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # ⭐️ 3개 클래스이므로 categorical 사용!
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# VGG16 모델 로드 (top 빼고 가져오기 → feature extractor 용도). Conv2D/MaxPooling 구조 전체를 가져와서 대신 쓰는 것
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# 모든 레이어를 고정 (feature extractor 용)
for layer in vgg_base.layers:
    layer.trainable = False

# 새 분류기 층 추가 (고양이, 개, 말 → 3개 클래스)
x = vgg_base.output
x = GlobalAveragePooling2D()(x)  # Flatten 대신 이거 써도 좋아
x = Dense(256, activation='relu')(x)
output_layer = Dense(3, activation='softmax')(x)  # ⭐️ 3개 클래스이므로 3 출력

# 새 모델 구성
model = Model(inputs=vgg_base.input, outputs=output_layer)

# 컴파일
model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 요약 출력
model.summary()

# 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)
